In [11]:
# Cell 1: Import Libraries

import argparse
import os
import re
import scipy.io as sio
import numpy as np
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, f1_score


In [12]:
# Cell 2: Define Constants

W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4
classes = ['W', 'N1', 'N2', 'N3', 'REM']
n_classes = len(classes)


In [13]:
# Cell 3: Define Functions

def evaluate_metrics(cm):
    cm = cm.astype(np.float32)
    FP = cm.sum(axis=0) - np.diag(cm)
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    PPV = TP / (TP + FP)
    NPV = TN / (TN + FN)
    FPR = FP / (FP + TN)
    FNR = FN / (TP + FN)
    FDR = FP / (TP + FP)

    ACC = (TP + TN) / (TP + FP + FN + TN)
    ACC_macro = np.mean(ACC)

    F1 = (2 * PPV * TPR) / (PPV + TPR)
    F1_macro = np.mean(F1)

    return ACC_macro, ACC, F1_macro, F1, TPR, TNR, PPV

def print_performance(cm, y_true=[], y_pred=[]):
    tp = np.diagonal(cm).astype(float)
    tpfp = np.sum(cm, axis=0).astype(float) 
    tpfn = np.sum(cm, axis=1).astype(float) 
    acc = np.sum(tp) / np.sum(cm)
    precision = tp / tpfp
    recall = tp / tpfn
    f1 = (2 * precision * recall) / (precision + recall)

    FP = cm.sum(axis=0).astype(float) - np.diag(cm)
    FN = cm.sum(axis=1).astype(float) - np.diag(cm)
    TP = np.diag(cm).astype(float)
    TN = cm.sum().astype(float) - (FP + FN + TP)
    specificity = TN / (TN + FP) 

    mf1 = np.mean(f1)

    overall_accuracy = np.mean(y_true == y_pred) if len(y_true) > 0 else acc

    return {
        'accuracy': overall_accuracy,
        'macro_f1': mf1
    }


In [14]:
# Cell 4: Define Performance Overall Function

def perf_overall(data_dir):
    allfiles = os.listdir(data_dir)
    outputfiles = [os.path.join(data_dir, f) for f in allfiles if re.match(r"^output_.+\d+\.npz", f)]
    outputfiles.sort()

    max_accuracy = -1
    max_accuracy_file = ""

    for fpath in outputfiles:
        with np.load(fpath) as f:
            if len(f["y_true"].shape) == 1:
                if len(f["y_true"]) < 10:
                    f_y_true = np.hstack(f["y_true"])
                    f_y_pred = np.hstack(f["y_pred"])
                else:
                    f_y_true = f["y_true"]
                    f_y_pred = f["y_pred"]
            else:
                f_y_true = f["y_true"].flatten()
                f_y_pred = f["y_pred"].flatten()

            cm = confusion_matrix(f_y_true, f_y_pred, labels=[0, 1, 2, 3, 4])
            perf = print_performance(cm, f_y_true, f_y_pred)

            if perf['accuracy'] > max_accuracy:
                max_accuracy = perf['accuracy']
                max_accuracy_file = fpath

    print(f"File with maximum accuracy: {max_accuracy_file}")
    print(f"Maximum accuracy: {max_accuracy:.4f}")

    y_true = []
    y_pred = []
    for fpath in outputfiles:
        with np.load(fpath) as f:
            if len(f["y_true"].shape) == 1:
                if len(f["y_true"]) < 10:
                    f_y_true = np.hstack(f["y_true"])
                    f_y_pred = np.hstack(f["y_pred"])
                else:
                    f_y_true = f["y_true"]
                    f_y_pred = f["y_pred"]
            else:
                f_y_true = f["y_true"].flatten()
                f_y_pred = f["y_pred"].flatten()

            y_true.extend(f_y_true)
            y_pred.extend(f_y_pred)

    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    sio.savemat('con_matrix_sleep.mat', {'y_true': y_true, 'y_pred': y_pred})
    cm = confusion_matrix(y_true, y_pred, labels=range(n_classes))
    acc = np.mean(y_true == y_pred)
    mf1 = f1_score(y_true, y_pred, average="macro")

    print("Overall Performance:")
    print(f"Maximum accuracy: {max_accuracy:.4f}")
    print(f"Macro-F1 score: {mf1:.4f}")


In [15]:
# Cell 5: Define Main Function and Execute

# In Jupyter Notebook, the command line arguments are not used.
# Instead, directly call the function with the desired directory path.

data_dir = "./outputs_2013/outputs_eeg_fpz_cz"  # Set your data directory here
perf_overall(data_dir)


File with maximum accuracy: ./outputs_2013/outputs_eeg_fpz_cz\output_fold6.npz
Maximum accuracy: 0.9022
Overall Performance:
Maximum accuracy: 0.9022
Macro-F1 score: 0.7966
